In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
historical_data = pd.read_csv("csv_processed/hashed_data.csv")
room_crosswalk= pd.read_csv("csv_processed/listing_name_crosswalk.csv")

In [3]:
print(historical_data.head())

   Unnamed: 0        Date         Type Confirmation Code  Start Date  Nights  \
0           0  03/04/2019       Payout               NaN         NaN     NaN   
1           1  03/04/2019  Reservation        HM3FARRJMQ  03/03/2019     5.0   
2           2  03/04/2019  Reservation        HMYHPEJZSQ  03/03/2019     3.0   
3           3  03/03/2019  Reservation        HMPANYMCWT  03/02/2019     2.0   
4           4  03/04/2019       Payout               NaN         NaN     NaN   

                                           Listing                Details  \
0                                              NaN  Transfer to *****4871   
1  Red Victorian - Private room w/sink, Tea Garden                    NaN   
2    Red Victorian - Private room w bath, Sunshine                    NaN   
3     Red Victorian - Private ensuite, Rose Garden                    NaN   
4                                              NaN  Transfer to *****6485   

   Reference Currency  Amount  Paid Out  Host Fee  Clean

In [4]:
# Add in room names
historical_data=historical_data.merge(room_crosswalk, how='left', on='Listing')

# Month and Year columns added for accounting purposes
historical_data['Month'] = historical_data.Date.apply(lambda dateCol: datetime.datetime.strptime(dateCol,'%m/%d/%Y').month)
historical_data['Year'] = historical_data.Date.apply(lambda dateCol: datetime.datetime.strptime(dateCol,'%m/%d/%Y').year)
data = historical_data[historical_data.Year == 2019][historical_data.Month < 3]

print(data.head())
print(data.dtypes)
print(data.Type.unique())

    Unnamed: 0        Date         Type Confirmation Code  Start Date  Nights  \
38          38  02/28/2019  Reservation        HM2Z3XNDB5  02/27/2019     2.0   
39          39  02/27/2019  Reservation        HMEX5E4HRK  02/26/2019     3.0   
40          40  02/24/2019  Reservation        HM5TXCTMRF  02/23/2019     1.0   
46          46  02/28/2019       Payout               NaN         NaN     NaN   
47          47  02/28/2019  Reservation        HMRRHYXTTJ  02/27/2019     1.0   

                                              Listing                Details  \
38       Red Victorian - Private room -  Rainbow Room                    NaN   
39  Red Victorian - Private suite w bath, Flower C...                    NaN   
40  Red Victorian - Private room w loftbed, Butterfly                    NaN   
46                                                NaN  Transfer to *****4871   
47       Red Victorian - Private ensuite, Rose Garden                    NaN   

    Reference Currency  ...  Pai

/Users/tobie/.local/share/virtualenvs/airbnb-analysis-P_6EU12G/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [5]:
# Total paid out by month
total = data[data.Type == 'Payout']
print(total.groupby('Month')['Paid Out'].sum())

Month
1    46617.88
2    44953.46
Name: Paid Out, dtype: float64


In [6]:
# Gross hotel revenue by listing
payouts = data[data.Type.isin(['Reservation','Adjustment'])].groupby(['Month','Listing']).agg({"Amount":np.sum}).reset_index()
print(payouts.dtypes)
print(payouts.head())
payouts.to_csv('csv_export/gross_data_2019.csv')

Month        int64
Listing     object
Amount     float64
dtype: object
   Month                                            Listing  Amount
0      1    Red Victorian - Air Bunk in shared Peacock Room  938.28
1      1        Red Victorian - Bed  in shared Friends Room  408.57
2      1  Red Victorian - Bunk #1 in shared Hole in the ...  238.62
3      1      Red Victorian - Bunk #1 in shared Poster Room  757.18
4      1  Red Victorian - Bunk #2 in shared Hole in the ...  569.20


In [7]:
# Total unassigned resolution adjustments by month
adjustments = data[data.Type.isin(['Resolution Adjustment','Resolution Payout'])].groupby('Month').agg({"Amount": np.sum})
print(adjustments)

       Amount
Month        
1     -283.33
2     -534.99


In [8]:
# Distribute unassigned resolution adjustments by month by rooms listed that month
rooms_per_month = payouts.groupby('Month').agg({"Listing": pd.Series.nunique})
print(rooms_per_month)

type(rooms_per_month)

monthly_distribution = pd.merge(adjustments, rooms_per_month, on='Month')
monthly_distribution['Amount Per Listing'] = (monthly_distribution.Amount / monthly_distribution.Listing).round(2)
print(monthly_distribution)

monthly_distribution = monthly_distribution.drop(['Amount', 'Listing'], axis=1)
print(monthly_distribution)

       Listing
Month         
1           32
2           29
       Amount  Listing  Amount Per Listing
Month                                     
1     -283.33       32               -8.85
2     -534.99       29              -18.45
       Amount Per Listing
Month                    
1                   -8.85
2                  -18.45


In [9]:
print(payouts)

    Month                                            Listing   Amount
0       1    Red Victorian - Air Bunk in shared Peacock Room   938.28
1       1        Red Victorian - Bed  in shared Friends Room   408.57
2       1  Red Victorian - Bunk #1 in shared Hole in the ...   238.62
3       1      Red Victorian - Bunk #1 in shared Poster Room   757.18
4       1  Red Victorian - Bunk #2 in shared Hole in the ...   569.20
5       1      Red Victorian - Bunk #2 in shared Poster Room   813.61
6       1      Red Victorian - Bunk #3 in shared Poster Room  1075.24
7       1      Red Victorian - Bunk #4 in shared Poster Room   728.31
8       1  Red Victorian - Earth Bunk in shared Peacock Room   887.05
9       1   Red Victorian - Fire Bunk in shared Peacock Room  1171.49
10      1       Red Victorian - Private ensuite, Rose Garden  3505.58
11      1       Red Victorian - Private room -  Rainbow Room  2913.53
12      1  Red Victorian - Private room 1/2 bath, Night Duty  2246.33
13      1   Red Vict

In [13]:
print(payouts)

    Month                                            Listing   Amount
0       1    Red Victorian - Air Bunk in shared Peacock Room   938.28
1       1        Red Victorian - Bed  in shared Friends Room   408.57
2       1  Red Victorian - Bunk #1 in shared Hole in the ...   238.62
3       1      Red Victorian - Bunk #1 in shared Poster Room   757.18
4       1  Red Victorian - Bunk #2 in shared Hole in the ...   569.20
5       1      Red Victorian - Bunk #2 in shared Poster Room   813.61
6       1      Red Victorian - Bunk #3 in shared Poster Room  1075.24
7       1      Red Victorian - Bunk #4 in shared Poster Room   728.31
8       1  Red Victorian - Earth Bunk in shared Peacock Room   887.05
9       1   Red Victorian - Fire Bunk in shared Peacock Room  1171.49
10      1       Red Victorian - Private ensuite, Rose Garden  3505.58
11      1       Red Victorian - Private room -  Rainbow Room  2913.53
12      1  Red Victorian - Private room 1/2 bath, Night Duty  2246.33
13      1   Red Vict

In [17]:
# Create new data frame to hold adjustments for listings
adjusted_payouts = payouts.merge(monthly_distribution, how='left', on='Month')
adjusted_payouts = adjusted_payouts.fillna(0)
adjusted_payouts['Adjusted Amount'] = adjusted_payouts.Amount + adjusted_payouts['Amount Per Listing']

adjusted_payouts = adjusted_payouts.drop('Amount Per Listing', axis=1)
print(payouts.columns.values)

# Round amounts to 2 decimal points for dollars and cents (0.00)
adjusted_payouts = adjusted_payouts.round(2)

# Sum by Room
adjusted_payouts_by_room=adjusted_payouts.merge(room_crosswalk, how="left", on="Listing")
adjusted_payouts_by_room=adjusted_payouts_by_room.groupby(["room-name","Month"]).agg({"Amount":np.sum}).reset_index()
adjusted_payouts_by_room=adjusted_payouts_by_room.round(2)
# Save adjusted payouts to new csv file
adjusted_payouts_by_room.to_csv('csv_export/adjusted_payouts_2019.csv')

['Month' 'Listing' 'Amount']
